In [347]:
import pandas as pd
import numpy as np
import os
import mzbatch
from sklearn.metrics.pairwise import cosine_similarity
import importlib as imp
imp.reload(mzbatch)
import subprocess
import warnings
warnings.filterwarnings("ignore") 

# Basic parameter

In [374]:
# analysis_batch, sepuzhu,pos_neg,filesampleid = "BloodNaoLiu240807","Amide","pos","QCidentify_2"
# analysis_batch, sepuzhu,pos_neg,filesampleid = "BloodNaoLiu240807","Amide","neg","QCidentify_2"
# analysis_batch, sepuzhu,pos_neg,filesampleid = "BloodNaoLiu240807","C18","pos","QCidentify_2"
# analysis_batch, sepuzhu,pos_neg,filesampleid = "BloodNaoLiu240807","C18","neg","QCidentify_2"
# analysis_batch,sepuzhu,pos_neg,filesampleid = "BloodYin240307","Amide","pos","QCidentify_1"
analysis_batch,sepuzhu,pos_neg,filesampleid = "BloodYin240307","Amide","neg","QCidentify_1"
# analysis_batch,sepuzhu,pos_neg,filesampleid = "BloodYin240307","C18","pos","QCidentify_1"
# analysis_batch,sepuzhu,pos_neg,filesampleid = "BloodYin240307","C18","neg","QCidentify_1"

analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
analysis_path


'BloodYin240307_Amide_neg'

In [ ]:
# Set diff parameters
standard_raw_diff = 15
standard_QC_diff = 15


In [377]:
root_path = f'../../Fulaoshi/230821_mzML/{analysis_path}/'
# middle_list = ['873_Amide','873_C18']
middle_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = f'../../Fulaoshi/230821_mzML_res/{analysis_path}'
standard13_ms2_path = '../../Fulaoshi/230821_mzML_res/rerun240410/'
ev13_list = [1,10,15,20,25,30,35,40,50,60,70,80,100]


In [ ]:
"""240920 Modify Reference"""
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all-revise20240920.xlsx", index_col=0, sheet_name="Sheet2")
ref_all_df['batch_sampleid'] = ref_all_df['batch'].astype(str) + '/' + ref_all_df['sampleid'].astype(str)
ref_all_df['bscp'] = ref_all_df['batch'] + '+' + ref_all_df['sampleid'].astype(str) + '+' + ref_all_df['CAS No.'].astype(str) + '+' + ref_all_df['pos_neg']
ref_all_df.head(2)


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid,ev_nums,ev_list
1104,873_C18,C18,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.1196,285.0,133000000,873_C18+26386+1867-73-8+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1106,873_C18,C18,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.2216,874.8,111000000,873_C18+26386+50-22-6+pos,[M+H]+,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."


In [380]:
ref_all_df.shape

(12484, 17)

In [ ]:
# Create 4 mz_list
Amide_pos_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "Amide") & (ref_all_df["pos_neg"] == "pos")]["m/z"].tolist()
Amide_pos_mzlist.sort()
Amide_neg_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "Amide") & (ref_all_df["pos_neg"] == "neg")]["m/z"].tolist()
Amide_neg_mzlist.sort()
C18_pos_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "C18") & (ref_all_df["pos_neg"] == "pos")]["m/z"].tolist()
C18_pos_mzlist.sort()
C18_neg_mzlist = ref_all_df[(ref_all_df["sepuzhu"] == "C18") & (ref_all_df["pos_neg"] == "neg")]["m/z"].tolist()
C18_neg_mzlist.sort()

# Create the upper and lower bounds for the 4 mz_lists
Ap_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=standard_raw_diff)] for i in Amide_pos_mzlist]
An_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=standard_raw_diff)] for i in Amide_neg_mzlist]
Cp_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=standard_raw_diff)] for i in C18_pos_mzlist]
Cn_mzrange_list = [[i, mzbatch.get_mz_range(i, diff=standard_raw_diff)] for i in C18_neg_mzlist]

mzrange_dict = {"Amide_pos": Ap_mzrange_list, "Amide_neg": An_mzrange_list, 
                "C18_pos": Cp_mzrange_list, "C18_neg": Cn_mzrange_list}


In [ ]:
"""Set mzrange_list"""
mzrange_list = mzrange_dict[f"{sepuzhu}_{pos_neg}"]
len(mzrange_list), f"{sepuzhu}_{pos_neg}"


(2715, 'Amide_neg')

In [385]:
mzML_filename_list =[ f"{root_path}/mzML/{filesampleid}-NCE{i}-{pos_neg}.mzML" for i in ev13_list]
pkl_filename_list = [f"{root_path}/pkl/{filesampleid}-NCE{i}-{pos_neg}.pkl" for i in ev13_list]
pkl_filter_path_list = [i.replace("/pkl/","/pkl_filter/") for i in pkl_filename_list]
mzML_filename_list[0], pkl_filename_list[0], pkl_filter_path_list[0]

('../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//mzML/QCidentify_1-NCE1-neg.mzML',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//pkl/QCidentify_1-NCE1-neg.pkl',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//pkl_filter/QCidentify_1-NCE1-neg.pkl')

In [ ]:
# Generate QC file list
QC_filename_list = [f"{root_path}/MS-DIAL/{filesampleid}-NCE{i}-{pos_neg}.txt" for i in ev13_list]
QC_filename_list


['../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE1-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE10-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE15-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE20-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE25-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE30-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE35-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE40-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE50-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE60-neg.txt',
 '../../Fulaoshi/230821_mzML/BloodYin240307_Amide_neg//MS-DIAL/QCidentify_1-NCE70

In [ ]:
# Filter raw files, only keep mz values in the 4000+ compounds list
for i in range(len(mzML_filename_list)):
    mzML_path = mzML_filename_list[i]
    pkl_path = pkl_filename_list[i]
    pkl_filter_folderpath = f"{root_path}/pkl_filter/"
    pkl_filter_path = pkl_filter_path_list[i]
    
    raw_list = mzbatch.get_TIC_from_File(mzML_path, pkl_path)
    raw_df = pd.DataFrame(raw_list)
    raw_ms2_df = raw_df[raw_df["MSlevel"] == 2]
    
    # Filter mz values in the 4000 compounds for MS2 data
    valid_index_list = []  # List to store valid indices
    for theo_mz, (mz_low, mz_up) in mzrange_list:
        temp_df = raw_ms2_df[(raw_ms2_df['pecursor'] >= mz_low) & (raw_ms2_df['pecursor'] <= mz_up)]
        if temp_df.shape[0] > 0:
            valid_index_list.extend(temp_df.index.tolist())
        # break
    valid_index_list = list(set(valid_index_list))
    valid_index_list.sort()
    
    # Output to file
    if not os.path.exists(pkl_filter_folderpath):
        os.makedirs(pkl_filter_folderpath)
        while os.path.exists(pkl_filter_folderpath):
            break
    raw_ms2_df.loc[valid_index_list, :].to_pickle(pkl_filter_path)
    print(raw_ms2_df.loc[valid_index_list, :].shape)


(5827, 9)
(5785, 9)
(6010, 9)
(5906, 9)
(5916, 9)
(5734, 9)
(5761, 9)
(5750, 9)
(5752, 9)
(5738, 9)
(5711, 9)
(5727, 9)
(5685, 9)


C18_pos 4681-5899
C18_neg 8249-8700

In [ ]:
# Put into a list
raw_filter_ms2_df_list = [pd.read_pickle(i) for i in pkl_filter_path_list]


In [ ]:
# Filter MS-DIAL files
# 240824 update: Merge 13 QC files
QC_df_list = []
for QC_filename in QC_filename_list[0:]:
    QC35_df = pd.read_csv(QC_filename, index_col=0, sep="\t")
    QC35_df = QC35_df.loc[:, ["RT left(min)", "RT right (min)", "Precursor m/z", "Area"]]
    QC35_df["RT left(s)"] = QC35_df["RT left(min)"] * 60
    QC35_df["RT right(s)"] = QC35_df["RT right (min)"] * 60
    QC_df_list.append(QC35_df.copy(deep=True))

con_QC_df = pd.concat(QC_df_list)
con_QC_df.reset_index(drop=True, inplace=True)
print(con_QC_df.shape)
con_QC_df.tail(5)
con_QC_df.sort_values(by=["Precursor m/z"]).head(10)


(129992, 6)


,RT left(min),RT right (min),Precursor m/z,Area,RT left(s),RT right(s)
66551,6.278,6.583,70.02985,9460,376.68,394.98
99946,20.587,20.909,70.35116,10324,1235.22,1254.54
4083,4.190,4.556,70.65295,14132,251.40,273.36
125495,5.726,5.814,70.94605,3069758,343.56,348.84
95740,5.838,6.291,70.94607,6222704,350.28,377.46
75618,5.371,5.799,70.94607,7426415,322.26,347.94
95477,5.358,5.821,70.94607,7816336,321.48,349.26
55783,5.869,6.335,70.94608,6407718,352.14,380.10
75921,5.799,6.304,70.94608,7484609,347.94,378.24
15379,5.455,5.799,70.94608,7870141,327.30,347.94


In [ ]:
# Filter QC35 based on mz, 240824 updated to filter con_QC
QC_valid_index_list = []  # List to store valid indices
theo_mz_list = []
for theo_mz, (mz_low, mz_up) in mzrange_list[0:]:
    temp_QC35_df = con_QC_df[(con_QC_df['Precursor m/z'] >= mz_low) & (con_QC_df['Precursor m/z'] <= mz_up)]
    if temp_QC35_df.shape[0] > 0:
        QC_valid_index_list.extend(temp_QC35_df.index.tolist())
        theo_mz_list.extend([theo_mz] * temp_QC35_df.shape[0])
    # break
con_QC_filter_df = con_QC_df.loc[QC_valid_index_list, :].copy(deep=True)
con_QC_filter_df["theo_mz"] = theo_mz_list
con_QC_filter_df.drop_duplicates(subset=['Precursor m/z', 'RT left(s)', 'RT right(s)', 'theo_mz'], inplace=True)
con_QC_filter_df.sort_values(by="theo_mz", inplace=True)
con_QC_filter_df.reset_index(inplace=True, drop=True)

# Output to file
if not os.path.exists(pkl_filter_folderpath):
    os.makedirs(pkl_filter_folderpath)
    while os.path.exists(pkl_filter_folderpath):
        break
con_QC_filter_df.to_pickle(f"{pkl_filter_folderpath}/con_QC_filter_df.pkl")
print(con_QC_filter_df.shape)
con_QC_filter_df.head(3), con_QC_filter_df.tail(3)


(13209, 7)


(   RT left(min)  RT right (min)  Precursor m/z     Area  RT left(s)  \
 0         7.317           7.932       73.02950  6344055      439.02   
 1         7.382           7.883       73.02950  5578152      442.92   
 2        11.110          11.682       73.02951  3567743      666.60   
 
    RT right(s)  theo_mz  
 0       475.92  73.0295  
 1       472.98  73.0295  
 2       700.92  73.0295  ,
        RT left(min)  RT right (min)  Precursor m/z     Area  RT left(s)  \
 13206         4.007           4.615      956.58478  1555547      240.42   
 13207         3.544           4.009      956.57910   841593      212.64   
 13208         3.887           4.404      964.57935   277208      233.22   
 
        RT right(s)    theo_mz  
 13206       276.90  956.57407  
 13207       240.54  956.57407  
 13208       264.24  964.56524  )

In [ ]:
# Construct the DataFrame to store the results
res_cols_list = ["theo_mz", "ref_mz", "ref_rt_left", "ref_rt_right", "Area", "ms2_ev", "ms2_index", "ms2_mz", "ms2_rt"]
print(res_cols_list[0:])


['theo_mz', 'ref_mz', 'ref_rt_left', 'ref_rt_right', 'Area', 'ms2_ev', 'ms2_index', 'ms2_mz', 'ms2_rt']


In [ ]:
# Align based on QC: find each energy mz within 15ppm, and rt within range
res_df = pd.DataFrame(columns=res_cols_list)
n = 0
for QC35_filter_index in con_QC_filter_df.index[0:]:  # QC35 is always 1
    theo_mz, ref_mz, ref_rt_left, ref_rt_right, Area = con_QC_filter_df.loc[QC35_filter_index, ['theo_mz', 'Precursor m/z', "RT left(s)", "RT right(s)", "Area"]]
    theo_mz_low, theo_mz_up = mzbatch.get_mz_range(theo_mz, diff=standard_QC_diff)
    
    for i, raw_filter_ms2_df in enumerate(raw_filter_ms2_df_list[0:]):
        ms2_ev = ev13_list[i]  # Use this energy level as the reference
        ms2_temp_df = raw_filter_ms2_df[((raw_filter_ms2_df['pecursor'] >= theo_mz_low) & (raw_filter_ms2_df['pecursor'] <= theo_mz_up)) 
                                        & ((raw_filter_ms2_df['RT'] >= ref_rt_left) & (raw_filter_ms2_df['RT'] <= ref_rt_right))].copy(deep=True)
        if ms2_temp_df.shape[0] > 0:
            for ms2_index in ms2_temp_df.index:
                ms2_mz, ms2_rt = ms2_temp_df.loc[ms2_index, ["pecursor", "RT"]]
                res_df.loc[n] = [theo_mz, ref_mz, ref_rt_left, ref_rt_right, Area, ms2_ev, ms2_index, ms2_mz, ms2_rt]
                n += 1
        # break


In [ ]:
res_df.to_csv(f"{out_root_path}/res_df_firstfilter.csv")

### Increase the first order intensity of each chart

In [ ]:
# Filter to get all MS1 spectra
raw_df_list = [pd.DataFrame(pd.read_pickle(i)) for i in pkl_filename_list]
raw_ms1_df_list = [raw_df[raw_df["MSlevel"] == 1] for raw_df in raw_df_list]


In [ ]:
ms1_intensity_list = []
for ref_index in res_df.index[0:]:
    ms2_ev, ms2_mz, ms2_rt = res_df.loc[ref_index,["ms2_ev","ms2_mz","ms2_rt"]]
    ms2_ev_index = ev13_list.index(ms2_ev)
    ms2_mz_low,ms2_mz_up = mzbatch.get_mz_range(ms2_mz)
    
    raw_ms1_df = raw_ms1_df_list[ms2_ev_index] 
    ms1_df = pd.DataFrame(raw_ms1_df[raw_ms1_df["RT"] < ms2_rt].iloc[-1,-2],columns=["Mass","Intensity"])
    ms1_intensity_filter_df  = ms1_df[(ms1_df['Mass'] >= ms2_mz_low) & (ms1_df['Mass'] <= ms2_mz_up)]
    if ms1_intensity_filter_df.shape[0]>0:
        ms1_intensity_list.append(ms1_intensity_filter_df["Intensity"].max())
    else:
        ms1_intensity_list.append(-1)


In [2062]:
len(ms1_intensity_list),ms1_intensity_list.count(-1)

(274324, 26060)

In [2063]:
res_df["ms1_intensity"] = ms1_intensity_list

In [ ]:
res_outpath = f"{out_root_path}/aligned_file.csv"
if not os.path.exists(os.path.dirname(res_outpath)):
    os.makedirs(os.path.dirname(res_outpath))
    while os.path.exists(os.path.dirname(res_outpath)):
        break
res_df.to_csv(res_outpath)


# Discard the -1 and keep only the diagram with the highest first-order strength
    

In [ ]:
# Drop entries with -1
res_nofu1_df = res_df[res_df["ms1_intensity"] != -1]
res_nofu1_df.shape


(248264, 10)

In [2070]:
max_ms1_indexes = res_nofu1_df.groupby(by=['theo_mz','ref_rt_left','ref_rt_right','ms2_ev'])['ms1_intensity'].idxmax()
res_maxms1_df = res_nofu1_df.loc[max_ms1_indexes,]
res_maxms1_df.shape

(84181, 10)

In [2071]:
res_groupby_df = res_maxms1_df.groupby(['theo_mz','ref_mz', 'ref_rt_left', 'ref_rt_right']).agg({
    'ms2_ev': list,
    'ms2_index': list
}).reset_index()
res_groupby_df['ms2_ev'] = res_groupby_df['ms2_ev'].apply(lambda x: [int(i) for i in x])
res_groupby_df['ms2_index'] = res_groupby_df['ms2_index'].apply(lambda x: [int(i) for i in x])

In [2073]:
res_groupby_df.to_csv(f"{out_root_path}/res_groupby_df.csv")

In [2172]:
res_groupby_df = pd.read_csv(f"{out_root_path}/res_groupby_df.csv",index_col=0)
res_groupby_df

,theo_mz,ref_mz,ref_rt_left,ref_rt_right,ms2_ev,ms2_index
0,72.99311,72.99302,44.16,62.64,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[801, 811, 847, 835, 821, 826, 846, 791, 836, ..."
1,72.99311,72.99305,40.44,61.68,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[801, 811, 847, 835, 821, 826, 846, 791, 836, ..."
2,72.99311,72.99306,40.74,66.18,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[801, 811, 847, 835, 821, 826, 846, 791, 836, ..."
3,72.99311,72.99306,41.16,60.90,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[801, 811, 847, 835, 821, 826, 846, 791, 836, ..."
4,72.99311,72.99307,40.50,63.30,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[801, 811, 847, 835, 821, 826, 846, 791, 836, ..."
...,...,...,...,...,...,...
7373,873.15198,873.14972,882.42,900.48,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 80, 100]","[13962, 13912, 13961, 13997, 13911, 13961, 139..."
7374,873.15198,873.14978,880.74,901.14,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 80, 100]","[13962, 13912, 13961, 13997, 13911, 13961, 139..."
7375,873.15198,873.14984,883.02,899.40,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 80, 100]","[13962, 13912, 13961, 13997, 13911, 13961, 139..."
7376,873.15198,873.15088,883.74,899.58,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 80, 100]","[13962, 13912, 13961, 13997, 13911, 13961, 139..."


# Add sample file

In [ ]:
"""Construct Area_cols_dict for Yin"""
def get_Area_cols_dict():
    Area_cols_dict = {}
    for temp_sepuzhu, temp_pos_neg in [['Amide', 'pos'], ['Amide', 'neg'], ['C18', 'pos'], ['C18', 'neg']]:
        MS_DIAL_list = os.listdir(f"../../Fulaoshi/230821_mzML/BloodYin240307_{temp_sepuzhu}_{temp_pos_neg}/MS-DIAL")
        Area13_filename_list = [i for i in MS_DIAL_list if i.startswith("Area")]
        Area13_df = pd.read_csv(f"../../Fulaoshi/230821_mzML/BloodYin240307_{temp_sepuzhu}_{temp_pos_neg}/MS-DIAL/{Area13_filename_list[0]}", sep="\t", index_col=0)
        Area_cols_list = [i for i in Area13_df.columns if ((i.strip().endswith(f"-{temp_pos_neg}") & (i.count('-') == 2) & (i.count('_') == 0)))]
        Area_cols_dict[f"{temp_sepuzhu}_{temp_pos_neg}"] = Area_cols_list
    return Area_cols_dict

Area_cols_dict = get_Area_cols_dict()
[print(i, len(v)) for i, v in Area_cols_dict.items()]


Amide_pos 111
Amide_neg 111
C18_pos 111
C18_neg 111


[None, None, None, None]

In [ ]:
# """Parameter changes for rerun on 240915"""
# # analysis_batch, sepuzhu, pos_neg, filesampleid = "BloodYin240307", "Amide", "pos", "QCidentify_1"
# # analysis_batch, sepuzhu, pos_neg, filesampleid = "BloodYin240307", "Amide", "neg", "QCidentify_1"
# analysis_batch, sepuzhu, pos_neg, filesampleid = "BloodYin240307", "C18", "pos", "QCidentify_1"
analysis_batch, sepuzhu, pos_neg, filesampleid = "BloodYin240307", "C18", "neg", "QCidentify_1"

analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"

root_path = f'../../Fulaoshi/230821_mzML/{analysis_path}/'
# middle_list = ['873_Amide', '873_C18']
middle_list = ['700_Amide', '700_C18', '873_Amide', '873_C18', '3545_Amide', '3545_C18', 'BC_Amide', 'BC_C18']
out_root_path = f'../../Fulaoshi/230821_mzML_res/{analysis_path}'
standard13_ms2_path = '../../Fulaoshi/230821_mzML_res/rerun240410/'
ev13_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 100]


In [299]:
MS_DIAL_list = os.listdir(f"{root_path}/MS-DIAL")
Area13_filename_list = [i for i in MS_DIAL_list if i.startswith("Area")]
root_path, Area13_filename_list

('../../Fulaoshi/230821_mzML/BloodYin240307_C18_neg/',
 ['Area_1_2024_08_27_13_13_16.txt'])

In [301]:
Area13_df = pd.read_csv(f"{root_path}/MS-DIAL/{Area13_filename_list[0]}", sep="\t",index_col=0)
Area13_df["Average Rt(s)"] = Area13_df["Average Rt(min)"]*60
Area13_df.head(2)

,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,Formula,...,QC-NCE204060-neg-15,QC-NCE204060-neg-16,QC-NCE204060-neg-17,QC-NCE204060-neg-18,QC-NCE204060-neg-7,QC-NCE204060-neg-8,QC-NCE204060-neg-9,1,1.1,Average Rt(s)
Alignment ID,,,,,,,,,,,,,,,,,,,,,
0,4.21,70.01294,Unknown,[M-H]-,NaN,0.01,False,NaN,NaN,NaN,...,0,4435,0,0,4276,3997,0,1304.424001,1507.990433,252.6
1,19.55,70.01971,no MS2: Oxazol,[M+H]+,NaN,0.01,False,-1.0,70.0287,C3H3NO,...,0,0,0,0,0,12559,0,2949.437947,3718.834778,1173.0


In [ ]:
# Area_cols_list = [i for i in Area13_df.columns if i.startswith("QCidentify")]
# Area_204060_colname = [i for i in Area_cols_list if '204060' in i][0]
# Area_cols_list.remove(Area_204060_colname)
# Change on 240817:
"""Brain tumor data"""
# Area_cols_list = [i for i in Area13_df.columns if ((i.strip().endswith(f"-{pos_neg}") & (i.count('-'))))]
"""Yin data"""
# Area_cols_list = [i for i in Area13_df.columns if ((i.strip().endswith(f"-{pos_neg}") & (i.count('-') == 2) & (i.count('_') == 0)))]
Area_cols_list = Area_cols_dict[f"{sepuzhu}_{pos_neg}"]

Area_lens = len(Area_cols_list)

len(Area_cols_list), Area_cols_list[0:3]


(111, ['20231225-10-neg', '20231225-11-neg', '20231225-12-neg'])

In [306]:
res_groupby_df = pd.read_csv(f"{out_root_path}/res_groupby_df.csv",index_col=0)

In [ ]:
Area13_3list = []
for res_groupby_index in res_groupby_df.index[0:]:
    res_groupby_mz, res_groupby_rt_low, res_groupby_rt_up = res_groupby_df.loc[res_groupby_index, ['ref_mz', 'ref_rt_left', 'ref_rt_right']] # Use theoretical mz as of 240915
    res_groupby_mz_low, res_groupby_mz_up = mzbatch.get_mz_range(res_groupby_mz, diff=10)
    
    # Filter Area13_df based on mz and rt ranges
    temp_Area13_df = Area13_df[
        ((Area13_df['Average Mz'] >= res_groupby_mz_low) & (Area13_df['Average Mz'] <= res_groupby_mz_up)) &
        ((Area13_df['Average Rt(s)'] >= res_groupby_rt_low) & (Area13_df['Average Rt(s)'] <= res_groupby_rt_up))
    ]
    
    if temp_Area13_df.shape[0] == 1:
        Area13_3list.append(temp_Area13_df.loc[:, Area_cols_list].values.tolist())
    elif temp_Area13_df.shape[0] > 1:
        # 240910 modification: retain all entries
        Area13_3list.append(temp_Area13_df.loc[:, Area_cols_list].values.tolist())
    else:
        Area13_3list.append(-1)


In [312]:
res_groupby_df["Area13_2list"] = Area13_3list

In [314]:
print(res_groupby_df.shape)
# res_groupby_df[res_groupby_df["Area13_2list"].apply(lambda x:x==-1)].head(2)

(8646, 7)


In [317]:
res_groupby_df.to_csv(f"{out_root_path}/res_groupby_Area13_01df.csv")

### Processing res_groupby_df

In [318]:
def fangda(x):
    if x=='-1' or x=='[[]]':
        return [[-1]*Area_lens]
    else:
        return x

In [319]:
# analysis_batch, sepuzhu,pos_neg = "BloodNaoLiu240807","Amide","pos"
# analysis_batch, sepuzhu,pos_neg = "BloodNaoLiu240807","Amide","neg"
# analysis_batch, sepuzhu,pos_neg = "BloodNaoLiu240807","C18","pos"
# analysis_batch, sepuzhu,pos_neg = "BloodNaoLiu240807","C18","neg"

# analysis_path = f"{analysis_batch}_{sepuzhu}_{pos_neg}"
# out_root_path = f'../../Fulaoshi/230821_mzML_res/{analysis_path}'

In [ ]:
# Process res_groupby_df
res_groupby_df = pd.read_csv(f"{out_root_path}/res_groupby_Area13_01df.csv", index_col=0)
res_groupby_df[res_groupby_df["Area13_2list"].apply(lambda x: x == '-1')].head(2)


,theo_mz,ref_mz,ref_rt_left,ref_rt_right,ms2_ev,ms2_index,Area13_2list
45,75.00876,75.00875,57.24,67.2,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[906, 897, 912, 902, 901, 908, 906, 896, 913, ...",-1
46,75.00876,75.00875,57.30,67.5,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[906, 897, 912, 902, 901, 908, 906, 896, 913, ...",-1


In [322]:
res_groupby_df["Area13_2list"] = res_groupby_df["Area13_2list"].apply(lambda x:fangda(x))
# res_groupby_df.to_csv(f"{out_root_path}/res_groupby_Area13_df.csv")

In [323]:
res_groupby_df[res_groupby_df["Area13_2list"].apply(lambda x:x==[[-1]*Area_lens])].head(2)

,theo_mz,ref_mz,ref_rt_left,ref_rt_right,ms2_ev,ms2_index,Area13_2list
45,75.00876,75.00875,57.24,67.2,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[906, 897, 912, 902, 901, 908, 906, 896, 913, ...","[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
46,75.00876,75.00875,57.30,67.5,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80...","[906, 897, 912, 902, 901, 908, 906, 896, 913, ...","[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."


In [326]:
res_groupby_df.to_csv(f"{out_root_path}/res_groupby_Area13_02df.csv")

In [327]:
res_groupby_df.shape

(8646, 7)

In [328]:
res_groupby_df["Area13_2list"] = res_groupby_df["Area13_2list"].apply(lambda x:eval(str(x)))

In [ ]:
# Split Area13_2list into multiple rows
res_groupby_df = res_groupby_df.explode("Area13_2list")


In [330]:
res_groupby_df.shape

(9203, 7)

In [331]:
res_groupby_df.reset_index(drop=True,inplace=True)

In [332]:
res_groupby_df.to_csv(f"{out_root_path}/res_groupby_Area13_03df.csv")


# Build merge13

- Separate the pluses and minuses

In [337]:
res_groupby_df = pd.read_csv(f"{out_root_path}/res_groupby_Area13_03df.csv",index_col=0)

In [ ]:
for res_groupby_index in res_groupby_df.index[0:]:
    ms2_ev_list, ms2_index_list = res_groupby_df.loc[res_groupby_index, ["ms2_ev", "ms2_index"]]
    ms2_ev_list = eval(ms2_ev_list)
    ms2_index_list = eval(ms2_index_list)
    
    con_ms2_df_list = []
    for i, ms2_ev in enumerate(ms2_ev_list):
        ms2_index = ms2_index_list[i]
        
        j = ev13_list.index(ms2_ev)
        raw_df = raw_df_list[j]
        one_ms2_df = pd.DataFrame(raw_df.loc[ms2_index, "peaks"], columns=["Mass", "Intensity"])
        one_ms2_df = mzbatch.round_df(one_ms2_df, 1e6)
        one_ms2_df.set_index(['Mass'], inplace=True)
        con_ms2_df_list.append(one_ms2_df)
    
    con_ms2_df = pd.concat(con_ms2_df_list, axis=1)
    con_ms2_cols = [int(i) for i in ms2_ev_list]
    con_ms2_df.columns = con_ms2_cols
    con_ms2_df = con_ms2_df.fillna(0)
    
    # Sort the dataframe by index
    con_ms2_df = con_ms2_df.sort_index()
    
    # Output to file
    merge13_outpath = f"{out_root_path}/NCE_merge13/"
    if not os.path.exists(merge13_outpath):
        os.makedirs(merge13_outpath)
        while os.path.exists(merge13_outpath):
            break
    
    con_ms2_df.to_csv(f"{merge13_outpath}/{res_groupby_index}.csv")


# Calculated according to merge13

In [ ]:
# Read standard graph
def read_bscp_merge13file(bscp):
    batch, sampleid, cas, pos_neg = bscp.split("+")
    filepath = f"{standard13_ms2_path}/{batch}/NCE_merge13/{bscp}.csv"
    # print(filepath)
    if not os.path.exists(filepath):
        return pd.DataFrame(columns=[-1])
    data_df = pd.read_csv(filepath, index_col=0)
    return data_df


In [93]:
def change_cols(data_df, ev_intersection_list, col_pre):
    data_df = data_df.loc[:,ev_intersection_list]
    data_cols = [f"{col_pre}_{i}" for i in data_df.columns]
    data_df.columns = data_cols
    return data_df

### Build the underlying data

In [ ]:
"""Parameter settings"""
sim_method_list = ["cosine", "jaccard", "entropy"]

curr_pattern = [sepuzhu, pos_neg]

# sim_method = sim_method_list[0]
sim_method = sim_method_list[1]
# sim_method = sim_method_list[2]


In [ ]:
"""Parameters related to ev"""
ev_13_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 100]
ev_0160_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60]
ev_0170_list = [1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70]
ev_2060_list = [20, 25, 30, 35, 40, 50, 60]
ev_204060_list = [20, 40, 60]
ev_select_dict = {"ev13": ev_13_list, "ev0160": ev_0160_list, "ev0170": ev_0170_list, "ev2060": ev_2060_list, "ev204060": ev_204060_list}

ev_select_name = "ev13"
ev_select_list = ev_select_dict[ev_select_name]


In [ ]:
"""Result column names"""
top_colist = []
[top_colist.extend([f"top_{i}", f"top_{i}_sum", f"top_{i}_ev13"]) for i in range(1, 301)]
print(len(top_colist))
align_res_cols = ["sepuzhu", "pos_neg", "bscp1", "b1_name", "b1_adduct", "b1_no0_linenums",
                  "b2_index", "b2_mz", "b2_rt_left", "b2_rt_right", "b2_no0_linenums",
                  "ev_nums", "ev_list", "Area13_2list"] + top_colist + Area_cols_list
print(len(align_res_cols), align_res_cols[0:3])


900
1025 ['sepuzhu', 'pos_neg', 'bscp1']


In [ ]:
"""Select reference corresponding to the pattern"""
pattern_ref_df = ref_all_df[(ref_all_df["sepuzhu"] == sepuzhu) & (ref_all_df["pos_neg"] == pos_neg)]
pattern_bscp_list = pattern_ref_df["bscp"].unique().tolist()
print(pattern_ref_df.shape)
pattern_ref_df.head(2)


(3139, 17)


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid,ev_nums,ev_list
1105,873_C18,C18,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,372.0,8140000,873_C18+26386+1867-73-8+neg,[M-H]-,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."
1107,873_C18,C18,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,874.8,921000,873_C18+26386+50-22-6+neg,[M-H]-,873_C18/26386,13,"[1, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80..."


### Start calculation

In [98]:
res_groupby_df.shape

(7707, 7)

#### Calculation loop

In [101]:
res_groupby_df.shape,out_root_path

((7707, 7), '../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_neg')

In [ ]:
import numpy as np

def cal_fragment(fragment_mzrange_list, b2_mzlist):
    # Convert the list to a NumPy array
    b2_mzarray = np.array(b2_mzlist)
    fragment_mzrange_array = np.array(fragment_mzrange_list) * 1e6
    # Separate the lower and upper limits of the range
    ilow = fragment_mzrange_array[:, 0]
    iup = fragment_mzrange_array[:, 1]
    # Use boolean indexing to find elements that meet the conditions
    mask = (b2_mzarray[:, np.newaxis] >= ilow) & (b2_mzarray[:, np.newaxis] <= iup)
    # Find all matching m/z values
    matched_indices = np.where(mask.any(axis=1))[0]
    fragment_mzlist = b2_mzarray[matched_indices]
    # fragment_mzlist = (fragment_mzlist * 1e6).astype(int).tolist()
    return fragment_mzlist

cal_fragment(fragment_mzrange_list, b2_mzlist)


In [ ]:
"""Add fragment filtering test"""
# Use b1's fragments to filter b2
filter_fragment_path = f"../../Fulaoshi/230821_mzML_res/rerun240410/{batch1}/NCE_filter_fragment/{bscp1}.csv"
if os.path.exists(filter_fragment_path):
    fragment_list = eval(pd.read_csv(filter_fragment_path, index_col=0)["filter_fragment"].tolist()[0])
# fragment_mzrange_list = [mzbatch.get_mz_range(i) for i in fragment_list]
fragment_mzrange_list = mzbatch.get_mz_range_matrix(fragment_list)

b2_mzlist = (b2_df.index / 1e6).tolist()
fragment_res_list = cal_fragment(fragment_mzrange_list, b2_mzlist)
# No fragments means no similarity calculation with b1
# if len(fragment_res_list) == 0:
#     continue


In [ ]:
# b1 contains the standard spectrum, b2 contains the spectrum of the actual sample
b1_align_res_df = pd.DataFrame(columns=align_res_cols) # Create a new DataFrame, reconstruction is too slow
for sim_method in sim_method_list[1:2]:

    for pattern_ref_index in pattern_ref_df.index[0:10]:

        # Basic data for b1
        b1_mz, b1_rt, bscp1, b1_name, b1_adduct = pattern_ref_df.loc[pattern_ref_index, ["m/z", "RT time", "bscp", "Product Name", "Adduct"]]
        batch1, sampleid1, cas1, pos_neg1 = bscp1.split("+")
        b1_mz_low, b1_mz_up = mzbatch.get_mz_range(b1_mz, diff=15)
        b1_align_res_df = b1_align_res_df.iloc[0:0]
        
        # Find b1 fragments
        filter_fragment_path = f"../../Fulaoshi/230821_mzML_res/rerun240410/{batch1}/NCE_filter_fragment/{bscp1}.csv"
        if not os.path.exists(filter_fragment_path):
            print(f"{bscp1} has no fragments")
            continue
        fragment_list = eval(pd.read_csv(filter_fragment_path, index_col=0)["filter_fragment"].tolist()[0])
        fragment_mzrange_list = mzbatch.get_mz_range_matrix(fragment_list, diff=15)

        # Reference for b2
        temp_res_df = res_groupby_df[((res_groupby_df['ref_mz'] >= b1_mz_low) & (res_groupby_df['ref_mz'] <= b1_mz_up))].copy(deep=True)
                                     # & ((res_groupby_df['ref_rt_right'] >= b1_rt) & (res_groupby_df['ref_rt_left'] <= b1_rt))].copy(deep=True)
        # Read b2
        n = 0
        for temp_res_index in temp_res_df.index[0:]:
            b2_mz, b2_rt_left, b2_rt_right, b2_Area13_2list = temp_res_df.loc[temp_res_index, ["ref_mz", "ref_rt_left", "ref_rt_right", "Area13_2list"]]
            if type(b2_Area13_2list) == str:
                b2_Area13_2list = eval(b2_Area13_2list)
            # Read data, first read b2, if no filtered b2 fragments, continue to the next loop
            b2_df = pd.read_csv(f"{out_root_path}/NCE_merge13/{temp_res_index}.csv", index_col=0)
            # Filter b2 fragments
            b2_mzlist = b2_df.index.tolist()
            # print(len(b2_mzlist))
            fragment_res_list = cal_fragment(fragment_mzrange_list, b2_mzlist)
            # If there are no fragments, do not calculate similarity with b1
            if len(fragment_res_list) == 0:
                # print(f"Unknown alignment graph {temp_res_index}, no common fragments, {len(fragment_res_list)}")
                continue
            # print(f"==== {bscp1} and unknown graph {temp_res_index} have common fragments, {len(fragment_res_list)}")
            
            # Filter b2 based on fragment results
            b2_df = b2_df.loc[fragment_res_list, :]
            b2_df["new_mass"] = np.around(np.array(fragment_res_list) / 1e4).astype(int)
            b2_df = b2_df.set_index("new_mass")
            b2_df = b2_df.groupby(b2_df.index).max()
            b2_df = mzbatch.to_relative_abundance(b2_df)
            b2_df = b2_df.mask(b2_df < 0.02, 0)
            b2_df = b2_df[(b2_df != 0).any(axis=1)]
            
            b1_df = read_bscp_merge13file(bscp1)
            b1_df = b1_df.mask(b1_df < 0.02, 0)
            b1_df = b1_df[(b1_df != 0).any(axis=1)]

            # ev select intersection
            ev1_list = [int(i) for i in b1_df.columns.tolist()]
            ev2_list = [int(i) for i in b2_df.columns.tolist()]
            ev_select_list = ev13_list
            ev_intersection_list = list(set(ev1_list) & set(ev2_list) & set(ev_select_list))
            ev_intersection_list.sort()
            if len(ev_intersection_list) == 0:
                continue
            ev_intersection_list = [str(i) for i in ev_intersection_list]
            b1_cols = [f"b1_{i}" for i in ev_intersection_list]  # Energy columns of b1
            b2_cols = [f"b2_{i}" for i in ev_intersection_list]  # Energy columns of b2

            b1_df = change_cols(b1_df, ev_intersection_list, "b1")
            b2_df = change_cols(b2_df, ev_intersection_list, "b2")
            # Merge b1 and b2
            b12_df = pd.concat([b1_df, b2_df], axis=1)
            b12_df = b12_df.fillna(0)
            b12_df = b12_df.sort_index()

            if b12_df.shape[0] == 0:
                print(f"b1: {bscp1}, b2: {bscp2} is empty")
                continue

            # Add group column, continuous differences of 1 are grouped together
            b12_df['group'] = (b12_df.index.to_series().diff() != 1).cumsum()
            b12_bak_df = b12_df.copy(deep=True)

            # Output the maximum value of continuous segments, if the maximum value is greater than 2, output the filename
            # if b12_df.groupby('group').size().max() > 2:
            #     print(f"Maximum continuous: {b12_df.groupby('group').size().max()}", b12_df.shape, f"{align_out_path}/{bscp2}.csv")
                # break

            # Take the maximum of each row
            b12_df = b12_df.groupby('group').agg(lambda x: x.max())
            # Remove the group column and set the minimum index value as the new index
            b12_df.index = b12_bak_df.groupby('group').apply(lambda x: x.index.min()).values

            # Calculate vertical similarity, otherwise it will cause an error
            if sim_method == "cosine":
                b12_df["cosine"] = np.square(np.diag(cosine_similarity(b12_df.loc[:, b1_cols].to_numpy(), b12_df.loc[:, b2_cols].to_numpy())))
            elif sim_method == "jaccard":
                b12_df["jaccard"] = np.diag(mzbatch.fu_jaccard_similarity_matrix(b12_df.loc[:, b1_cols].to_numpy(), b12_df.loc[:, b2_cols].to_numpy()))
            elif sim_method == "entropy":
                b12_df["entropy"] = np.diag(mzbatch.entropy_similarity(b12_df.loc[:, b1_cols].to_numpy(), b12_df.loc[:, b2_cols].to_numpy()))

            # Output path for merging energy files
            align_out_path = f"{out_root_path}/{batch1}/NCE_align_{sim_method}_{ev_select_name}/{bscp1}"
            if not os.path.exists(align_out_path):
                os.makedirs(align_out_path)
                while os.path.exists(align_out_path):
                    break

            # Calculate the summed intensity of b1 and b2
            b12_df["b1_sum_intensity"] = b12_df.loc[:, b1_cols].sum(axis=1)
            b12_df["b2_sum_intensity"] = b12_df.loc[:, b2_cols].sum(axis=1)
            b12_df["b12_sum_intensity"] = b12_df["b1_sum_intensity"] + b12_df["b2_sum_intensity"]
            b12_df.to_csv(f"{align_out_path}/{temp_res_index}.csv")

            """Output the vertical similarity results to NCE_align_res"""
            b12_no0_df = b12_df[~(b12_df[b1_cols] == 0).all(axis=1)]  # Filter rows where all 13 columns of b1 are not 0
            b12_no0_sort_df = b12_no0_df.sort_values(by="b12_sum_intensity", ascending=False)  # Sort by summed intensity column, take top
            # b1_no0_linenums = b12_no0_sort_df.shape[0]

            b2_no0_df = b12_df[~(b12_df[b2_cols] == 0).all(axis=1)]
            # b2_no0_linenums = b2_no0_df.shape[0]
            b1_no0_linenums = b12_df[b12_df[b1_cols].sum(axis=1) != 0].shape[0]
            b2_no0_linenums = b12_df[b12_df[b2_cols].sum(axis=1) != 0].shape[0]
            # print(b1_no0_linenums, b2_no0_linenums)

            top_res_list = []
            # There are only top


## Merge files

In [70]:
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
sim_method_list = ["cosine", "jaccard", "entropy"][:]
ev_type_list = ['ev13', 'ev204060']
# out_root_path_list = ['../../Fulaoshi/230821_mzML_res/BloodNaoLiu240807_Amide_pos','../../Fulaoshi/230821_mzML_res/BloodNaoLiu240807_Amide_neg',
#                      '../../Fulaoshi/230821_mzML_res/BloodNaoLiu240807_C18_pos','../../Fulaoshi/230821_mzML_res/BloodNaoLiu240807_C18_neg']
out_root_path_list = ['../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_pos/','../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_neg/',
                     '../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_pos/','../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_neg/']

In [71]:
import itertools
combine_file_list = list(itertools.product(out_root_path_list,sim_method_list[0:],ev_type_list[0:1] ))
# [print(i,v) for i,v in enumerate(combine_file_list)]
print(len(combine_file_list))
for i,v in enumerate(combine_file_list):
    print(i,v)


12
0 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_pos/', 'cosine', 'ev13')
1 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_pos/', 'jaccard', 'ev13')
2 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_pos/', 'entropy', 'ev13')
3 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_neg/', 'cosine', 'ev13')
4 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_neg/', 'jaccard', 'ev13')
5 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_Amide_neg/', 'entropy', 'ev13')
6 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_pos/', 'cosine', 'ev13')
7 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_pos/', 'jaccard', 'ev13')
8 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_pos/', 'entropy', 'ev13')
9 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_neg/', 'cosine', 'ev13')
10 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_neg/', 'jaccard', 'ev13')
11 ('../../Fulaoshi/230821_mzML_res/BloodYin240307_C18_neg/', 'entropy', 'ev13')


#### Merge to the batch directory

In [ ]:
# Merge 01 to each batch
for i_path, sim_method, ev_select in combine_file_list[10:11]:
    for batch in batch_list:
        # Result file
        align_res_outpath = f'{i_path}/{batch}/NCE_align_res_{sim_method}_{ev_select}'
        align_res_df_list = []
        try:
            for align_res_filename in os.listdir(align_res_outpath):
                one_align_res_df = pd.read_csv(f'{align_res_outpath}/{align_res_filename}', index_col=0)
                align_res_df_list.append(one_align_res_df)
        except Exception as e:
            continue

        align_res_all_df = pd.concat(align_res_df_list)
        # similar_all_df.to_excel(f'{i_path}/{batch}/similar_all_df.xlsx')
        align_res_all_df.to_csv(f'{i_path}/{batch}/align_res_{sim_method}_{ev_select}_all_df.csv')


#### Merge files under each batch into the total directory

In [121]:
for i_path,sim_method,ev_select in combine_file_list[9:10]:
    batch_align_res_df_list = []
    for batch in batch_list:
        try:
            batch_align_res_df = pd.read_csv(f'{i_path}/{batch}/align_res_{sim_method}_{ev_select}_all_df.csv', index_col=0)
        except Exception as e:
            continue
        batch_align_res_df_list.append(batch_align_res_df)

    if batch_align_res_df_list==[]:
        continue
    con_align_res_df = pd.concat(batch_align_res_df_list)
    # con_align_res_df.to_csv(f'{i_path}/con_align_res_{sim_method}_{ev_select}_all_df.csv')

In [124]:
duplicates_colist = [col for col in con_align_res_df.columns if not col.startswith("b2_")]

In [ ]:
quchong_df = con_align_res_df.drop_duplicates(subset=duplicates_colist)
quchong_df.to_csv(f'{i_path}/deduplicated_con_align_res_{sim_method}_{ev_select}_all_df.csv')


# Calculate horizontal similarity

In [6]:
align_out_path = f"{out_root_path}/{batch1}/NCE_align_{sim_method}_{ev_select_name}/{bscp1}"

In [36]:
base_cols =["sepuzhu", "pos_neg", 
               "bscp1", "b1_name", "b1_adduct", 
               "b2_index", "_mz", "b2_rt_left", "b2_rt_right"]
con_sim_df = pd.DataFrame(columns=["sepuzhu", "pos_neg", 
                                   "bscp1", "b1_name", "b1_adduct", 
                                   "b2_index", "b2_mz", "b2_rt_left", "b2_rt_right", "b2_Area13_2list",
                                   'b1_1', 'b1_10', 'b1_15', 'b1_20', 'b1_25', 'b1_30', 
                                   'b1_35', 'b1_40', 'b1_50', 'b1_60', 'b1_70', 'b1_80', 'b1_100'])

In [37]:
import itertools
l1_list = ["Amide","C18"]
l2_list = ["pos","neg"]
l3_list = ["cosine","jaccard","entropy"]
l4_list = ['ev13', 'ev204060'][0:1]
heng_combine_list = list(itertools.product(l1_list, l2_list, l3_list,l4_list))
[print(i,v) for i,v in enumerate(heng_combine_list)]
len(heng_combine_list)

0 ('Amide', 'pos', 'cosine', 'ev13')
1 ('Amide', 'pos', 'jaccard', 'ev13')
2 ('Amide', 'pos', 'entropy', 'ev13')
3 ('Amide', 'neg', 'cosine', 'ev13')
4 ('Amide', 'neg', 'jaccard', 'ev13')
5 ('Amide', 'neg', 'entropy', 'ev13')
6 ('C18', 'pos', 'cosine', 'ev13')
7 ('C18', 'pos', 'jaccard', 'ev13')
8 ('C18', 'pos', 'entropy', 'ev13')
9 ('C18', 'neg', 'cosine', 'ev13')
10 ('C18', 'neg', 'jaccard', 'ev13')
11 ('C18', 'neg', 'entropy', 'ev13')


12

In [39]:
top_colist=[]
[top_colist.extend([f"top_{i}",f"top_{i}_sum",f"top_{i}_ev13"]) for i in range(1,301)]
print(len(top_colist))
align_res_cols = ["sepuzhu","pos_neg", "bscp1","b1_name","b1_adduct","b1_no0_linenums",
                  "b2_index","b2_mz","b2_rt_left","b2_rt_right","b2_no0_linenums",
                  "ev_nums","ev_list","Area13_2list"] + top_colist + Area_cols_list

900


In [ ]:
# b1 stores concatenated spectra, b2 stores actual sample spectra
for heng_sepuzhu, heng_pos_neg, heng_sim_method, heng_ev_select in heng_combine_list[7:8]:
    sim_df_list = []
    res_groupby_df_path = f'../../Fulaoshi/230821_mzML_res/{analysis_batch}_{heng_sepuzhu}_{heng_pos_neg}/res_groupby_Area13_03df.csv'
    res_groupby_df = pd.read_csv(res_groupby_df_path, index_col=0)
    pattern_ref_df = ref_all_df[(ref_all_df["sepuzhu"] == heng_sepuzhu) & (ref_all_df["pos_neg"] == heng_pos_neg)]
    Area_cols_list = Area_cols_dict[f"{heng_sepuzhu}_{heng_pos_neg}"]
    align_res_cols = ["sepuzhu", "pos_neg", "bscp1", "b1_name", "b1_adduct", "b1_no0_linenums",
                      "b2_index", "b2_mz", "b2_rt_left", "b2_rt_right", "b2_no0_linenums",
                      "ev_nums", "ev_list", "Area13_2list"] + top_colist + Area_cols_list
    
    n = 0
    j = 0
    for pattern_ref_index in pattern_ref_df.index[0:]:
        # b1 basic data
        b1_mz, b1_rt, bscp1, b1_name, b1_adduct = pattern_ref_df.loc[pattern_ref_index, ["m/z", "RT time", "bscp", "Product Name", "Adduct"]]
        batch1, sampleid1, cas1, pos_neg1 = bscp1.split("+")
        b1_mz_low, b1_mz_up = mzbatch.get_mz_range(b1_mz, diff=15)
        b1_align_res_df = pd.DataFrame(columns=align_res_cols)

        # b2 reference
        temp_res_df = res_groupby_df[((res_groupby_df['ref_mz'] >= b1_mz_low) & (res_groupby_df['ref_mz'] <= b1_mz_up))].copy(deep=True)
        # Only proceed if there are corresponding results
        if temp_res_df.shape[0] > 0:
            j += 1
            # print("111==", pattern_ref_index, temp_res_df.shape, j)
            # Read b2 data
            n = 0
            for temp_res_index in temp_res_df.index[0:]:
                b2_mz, b2_rt_left, b2_rt_right, b2_Area13_2list = temp_res_df.loc[temp_res_index, ["ref_mz", "ref_rt_left", "ref_rt_right", "Area13_2list"]]
                b2_Area13_2list = eval(b2_Area13_2list)
                # Read alignment data
                align_filepath = f'../../Fulaoshi/230821_mzML_res/{analysis_batch}_{heng_sepuzhu}_{heng_pos_neg}/{batch1}/NCE_align_{heng_sim_method}_{heng_ev_select}/{bscp1}/{temp_res_index}.csv'
                try:
                    align_file_df = pd.read_csv(align_filepath, index_col=0)
                except Exception as e:
                    # print(temp_res_index, bscp1, temp_res_index, e)
                    continue
                
                align_file_df = align_file_df.iloc[:, 0:-4]
                b1_cols = [i for i in align_file_df.columns if i.startswith("b1_")]
                b2_cols = [i for i in align_file_df.columns if i.startswith("b2_")]
                
                # Calculate similarity
                if heng_sim_method == "cosine":
                    pos_sim_list = np.square(np.diag(cosine_similarity(align_file_df.loc[:, b1_cols].T.to_numpy(), align_file_df.loc[:, b2_cols].T.to_numpy())))
                elif heng_sim_method == "jaccard":
                    pos_sim_list = np.diag(mzbatch.fu_jaccard_similarity_matrix(align_file_df.loc[:, b1_cols].T.to_numpy(), align_file_df.loc[:, b2_cols].T.to_numpy()))
                elif heng_sim_method == "entropy":
                    pos_sim_list = np.diag(mzbatch.entropy_similarity(align_file_df.loc[:, b1_cols].T.to_numpy(), align_file_df.loc[:, b2_cols].T.to_numpy()))
                
                pos_sim_list = list(pos_sim_list)
                base_list = [heng_sepuzhu, heng_pos_neg, bscp1, b1_name, b1_adduct, temp_res_index, b2_mz, b2_rt_left, b2_rt_right]
                sim_df_list.append(pd.DataFrame([base_list + pos_sim_list + b2_Area13_2list], columns=(base_cols + b1_cols + Area_cols_list)))
    
    if sim_df_list == []:
        continue
    # pd.concat(sim_df_list).to_csv(f"../../Fulaoshi/230821_mzML_res/{analysis_batch}_{heng_sepuzhu}_{heng_pos_neg}/con_similar_{heng_sim_method}_{heng_ev_select}.csv")
